In [5]:
from datasets import load_dataset

In [8]:
from datasets import load_dataset
imdb = load_dataset("imdb")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Reusing dataset imdb (/home/datngo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/datngo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-d9f4b46f288630b2.arrow
Loading cached processed dataset at /home/datngo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-b301346162b145e5.arrow


In [12]:
tokenized_imdb['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [7]:
rawsets = load_dataset('papluca/language-identification')
id2label = sorted(set(rawsets['train']['labels']))

Using custom data configuration papluca--language-identification-49176cc02800e1fc
Reusing dataset csv (/home/datngo/.cache/huggingface/datasets/papluca___csv/papluca--language-identification-49176cc02800e1fc/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

In [ ]:
def preprocess_func(examples):
    tokenized_example = tokenizer(examples['text'], truncation=True)
    tokenized_example['label'] = [id2label.index(l) for l in examples['labels']]
    return tokenized_example

In [ ]:
tokenized_sets = rawsets.map(preprocess_func, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=len(id2label))

In [ ]:
from kheiron import Trainer, TrainingOptions

options = TrainingOptions(task='text-classification',
                          train_batch_size=32,
                          eval_batch_size=32,
                          metric_for_best_model='macro_f1',
                          greater_is_better=True,)

In [ ]:
trainer = Trainer(model=model,
                  args=options,
                  train_set=tokenized_imdb['train'],
                  eval_set=tokenized_imdb['test'],
                  collate_fn=data_collator)

In [ ]:
trainer.train()